In [ ]:
!pip install pandas

In [20]:
import pandas as pd
from datetime import datetime, timedelta, date
from sklearn.cluster import KMeans

##Read the data and filter out data that belongs to country other than UK
retail_data = pd.read_csv('OnlineRetail.csv', encoding= 'unicode_escape')
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], errors = 'coerce')
uk_data = retail_data.query("Country=='United Kingdom'").reset_index(drop=True)

In [21]:
t1 = pd.Timestamp("2011-09-01 00:00:00.054000")
t2 = pd.Timestamp("2011-03-01 00:00:00.054000")
t3 = pd.Timestamp("2011-12-01 00:00:00.054000")
uk_data_6m = uk_data[(uk_data.InvoiceDate < t1) & (uk_data.InvoiceDate >= t2)].reset_index(drop=True)
uk_data_next = uk_data[(uk_data.InvoiceDate >= t1) & (uk_data.InvoiceDate < t3)].reset_index(drop=True)

In [22]:
uk_day_order = uk_data_6m[['CustomerID','InvoiceDate']]#convert Invoice Datetime to day
uk_day_order['InvoiceDay'] = uk_data_6m['InvoiceDate'].dt.date
uk_day_order = uk_day_order.sort_values(['CustomerID','InvoiceDate'])#drop duplicates
uk_day_order = uk_day_order.drop_duplicates(subset=['CustomerID','InvoiceDay'],keep='first')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
uk_day_order['PrevInvoiceDate'] = uk_day_order.groupby('CustomerID')['InvoiceDay'].shift(1)
uk_day_order['T2InvoiceDate'] = uk_day_order.groupby('CustomerID')['InvoiceDay'].shift(2)
uk_day_order['T3InvoiceDate'] = uk_day_order.groupby('CustomerID')['InvoiceDay'].shift(3)

In [24]:
uk_day_order.head()

,CustomerID,InvoiceDate,InvoiceDay,PrevInvoiceDate,T2InvoiceDate,T3InvoiceDate
649,12747.0,2011-03-01 14:53:00,2011-03-01,NaN,NaN,NaN
65091,12747.0,2011-05-05 15:31:00,2011-05-05,2011-03-01,NaN,NaN
90473,12747.0,2011-05-25 09:57:00,2011-05-25,2011-05-05,2011-03-01,NaN
124699,12747.0,2011-06-28 10:06:00,2011-06-28,2011-05-25,2011-05-05,2011-03-01
184410,12747.0,2011-08-22 10:38:00,2011-08-22,2011-06-28,2011-05-25,2011-05-05


In [25]:
uk_day_order['DayDiff'] = (uk_day_order['InvoiceDay'] - uk_day_order['PrevInvoiceDate']).dt.days
uk_day_order['DayDiff2'] = (uk_day_order['InvoiceDay'] - uk_day_order['T2InvoiceDate']).dt.days
uk_day_order['DayDiff3'] = (uk_day_order['InvoiceDay'] - uk_day_order['T3InvoiceDate']).dt.days

In [26]:
uk_day_diff = uk_day_order.groupby('CustomerID').agg({'DayDiff': ['mean','std']}).reset_index()
uk_day_diff.columns = ['CustomerID', 'DayDiffMean','DayDiffStd']

In [27]:
uk_day_order_last = uk_day_order.drop_duplicates(subset=['CustomerID'],keep='last')

In [28]:
uk_day_order_last = uk_day_order_last.dropna()
uk_day_order_last = pd.merge(uk_day_order_last, uk_day_diff, on='CustomerID')
uk_day_order_last = uk_day_order_last[['CustomerID','DayDiff','DayDiff2','DayDiff3','DayDiffMean','DayDiffStd']]
uk_day_order_last.columns = ['customerid','daydiff','daydiff2','daydiff3','daydiffmean','daydiffstd']

In [29]:
uk_day_order_last.head()

,customerid,daydiff,daydiff2,daydiff3,daydiffmean,daydiffstd
0,12747.0,55.0,89.0,109.0,43.500000,20.305993
1,12748.0,5.0,6.0,13.0,3.723404,3.083632
2,12749.0,7.0,17.0,87.0,25.000000,30.099834
3,12830.0,7.0,22.0,37.0,12.333333,4.618802
4,12839.0,20.0,44.0,70.0,32.800000,26.166773
